In [1]:
%load_ext autoreload
%autoreload 2
import time
from datetime import datetime, date, time, timedelta
import asyncio
import nest_asyncio
import importlib
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
from tastytrade import Account, Session, DXLinkStreamer
from tastytrade.dxfeed import Quote, Candle
from tastytrade.instruments import *
from config import *
from alphas import *

nest_asyncio.apply()

In [2]:
import tastytrade.instruments
importlib.reload(tastytrade.instruments)

<module 'tastytrade.instruments' from 'C:\\Users\\warre\\PycharmProjects\\tastytrade-bot\\.venv\\Lib\\site-packages\\tastytrade\\instruments.py'>

In [3]:
session=None
account=None

async def setup():
    global session, account
    # setup
    config = Config(test=False)
    session = Session(config.username, config.password, is_test=config.test)
    account = await Account.a_get(session, config.account_number)

    # account info
    balance = account.get_balances(session)
    positions = account.get_positions(session)
    history = account.get_history(session)

asyncio.run(setup())

In [4]:
streamer = DXLinkStreamer(session) #async streamer

SPXoptionchain = get_option_chain(session, "SPX")
VIXoptionchain = get_option_chain(session, "VIX")

SPXexpiries = SPXoptionchain.keys()
VIXexpiries = VIXoptionchain.keys()
#print(SPXexpiries)

In [5]:
VXfutures = Future.get(session, symbols = None, product_codes=["VX"])
VXfm = VXfutures[-1]
Vxbm = VXfutures[-2]

In [6]:
VXfm

Future(instrument_type=<InstrumentType.FUTURE: 'Future'>, symbol='/VXK25', product_code='VX', tick_size=Decimal('0.05'), notional_multiplier=Decimal('1000.0'), display_factor=Decimal('1.0'), last_trade_date=datetime.date(2025, 5, 21), expiration_date=datetime.date(2025, 5, 21), closing_only_date=datetime.date(2025, 5, 21), active=True, active_month=True, next_active_month=False, is_closing_only=False, stops_trading_at=datetime.datetime(2025, 5, 21, 13, 30, tzinfo=TzInfo(UTC)), expires_at=datetime.datetime(2025, 5, 21, 13, 30, tzinfo=TzInfo(UTC)), product_group='CFE', exchange='CFE', streamer_exchange_code='XCBF', back_month_first_calendar_symbol=True, streamer_symbol='/VXK25:XCBF', is_tradeable=True, future_product=FutureProduct(root_symbol='/VX', code='VX', description='Volatility Index', exchange='CFE', product_type='Financial', listed_months=[<FutureMonthCode.JAN: 'F'>, <FutureMonthCode.FEB: 'G'>, <FutureMonthCode.MAR: 'H'>, <FutureMonthCode.APR: 'J'>, <FutureMonthCode.MAY: 'K'>, <F

In [26]:
req = {
    "equity": ["AAPL"],
    "equity-option": ["AAPL  250620C00180000"],
    "cryptocurrency": ["BTC/USD"],
    "future": ["/ESM5"]
}
r = get_market_data_by_type(session,req)
print(r['items'])

[{'symbol': 'BTC/USD', 'instrument-type': 'Cryptocurrency', 'updated-at': '2025-04-25T06:29:31.118Z', 'bid': '93017.47', 'bid-size': '0.11516167', 'ask': '93971.74', 'ask-size': '0.0', 'mid': '93494.605', 'mark': '93494.605', 'last': '93494.61', 'last-mkt': '93494.61', 'open': '93089.28', 'day-high-price': '93497.265', 'day-low-price': '93047.21', 'close-price-type': 'Regular', 'prev-close': '93090.695', 'prev-close-price-type': 'Regular', 'summary-date': '2025-04-25', 'prev-close-date': '2025-04-24', 'is-trading-halted': False, 'trading-halted-reason': '', 'halt-start-time': -1, 'halt-end-time': -1, 'year-low-price': '49149.415', 'year-high-price': '109558.42', 'volume': '0.0'}, {'symbol': 'AAPL', 'instrument-type': 'Equity', 'updated-at': '2025-04-24T23:59:57.958Z', 'bid': '207.91', 'bid-size': '4.0', 'ask': '208.0', 'ask-size': '1.0', 'mid': '207.955', 'mark': '207.9991', 'last': '207.9991', 'last-ext': '221.8511', 'last-mkt': '208.37', 'beta': '1.265958606', 'dividend-amount': '0.2